### Importamos las librerías

In [1]:
import os
import time
import datetime
import numpy as np
import random
from tqdm import tqdm
import torch
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import AdamW, get_linear_schedule_with_warmup

### Uso de CPU ó GPU

In [2]:
# Establecer el valor de la semilla en todos lados para hacerlo reproducible.
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)

if torch.cuda.is_available():
    print("Usar GPU")
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)
    # Indicar a PyTorch que ejecute este modelo en la GPU.
    device = torch.device("cuda")
    batch_size = 1

else:
    print("Usar CPU")
    device = torch.device("cpu")
    batch_size = 1

Usar GPU


### Cargamos el Modelo de HuggingFace 

In [3]:
# Cargar el tokenizador de GPT.
tokenizer = AutoTokenizer.from_pretrained("flax-community/gpt-2-spanish", bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>')
model = AutoModelForCausalLM.from_pretrained("flax-community/gpt-2-spanish")
control_code = "ibai"
special_tokens_dict = {
         "additional_special_tokens": ['f"<|{control_code}|>"'],
}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
model.resize_token_embeddings(len(tokenizer))
unk_tok_emb = model.transformer.wte.weight.data[tokenizer.unk_token_id, :]
for i in range(num_added_toks):
        model.transformer.wte.weight.data[-(i+1), :] = unk_tok_emb

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


### Cargamos el Dataset “Ibai_textos.txt”

In [4]:
class GPT2Dataset(Dataset):
  def __init__(self, control_code, tokenizer, archivo_texto = 'ibai_textos.txt', max_length=768):
    self.tokenizer = tokenizer
    self.input_ids = []
    self.attn_masks = []
    print('loading text...')
    sentences = open(archivo_texto, 'r', encoding="utf-8").read().lower().split('\n')
    print('qty:',len(sentences))
    for row in tqdm(sentences):
      encodings_dict = tokenizer('<|startoftext|>'+ f"<|{control_code}|>" + row + '<|endoftext|>', truncation=True, max_length=max_length, padding="max_length")
      self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
      self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))
    
  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, idx):
    return self.input_ids[idx], self.attn_masks[idx] 

In [5]:
dataset = GPT2Dataset(control_code, tokenizer, archivo_texto="ibai_textos.txt", max_length=768)

# Dividir en conjuntos de entrenamiento y validación
train_size = int(0.99 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} muestras de entrenamiento'.format(train_size))
print('{:>5,} muestras de validación'.format(val_size))

train_dataloader = DataLoader(
            train_dataset,  # Las muestras de entrenamiento.
            sampler = RandomSampler(train_dataset), # Selecciona lotes de forma aleatoria.
            batch_size = batch_size # Entrena con este tamaño de lote.
        )

loading text...
qty: 10292


100%|██████████| 10292/10292 [00:03<00:00, 2846.39it/s]


10,189 muestras de entrenamiento
  103 muestras de validación


### Entrenamos haciendo el Fine-Tuning

In [6]:
epochs = 1
learning_rate = 5e-4
warmup_steps = 1e2
epsilon = 1e-8
sample_every = 500
optimizer = AdamW(model.parameters(),
                  lr = learning_rate,
                  eps = epsilon
                )
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = warmup_steps, 
                                            num_training_steps = total_steps)

def format_time(elapsed):
    return str(datetime.timedelta(seconds=int(round((elapsed)))))


C:\Users\Diego\anaconda3\envs\gpu\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


### Entrenar el modelo GPT-2 con nuestro Dataset

In [7]:

total_t0 = time.time()
model = model.to(device)
for epoch_i in range(0, epochs):
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')
    t0 = time.time()
    total_train_loss = 0
    model.train()
    for step, batch in enumerate(train_dataloader):
        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)
        model.zero_grad()
        outputs = model(  b_input_ids, labels=b_labels, 
                          attention_mask = b_masks, token_type_ids=None )
        loss = outputs[0]
        batch_loss = loss.item()
        total_train_loss += batch_loss
        # Get sample every x batches.
        if step % sample_every == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}. Loss: {:>5,}.   Elapsed: {:}.'.format(step, len(train_dataloader), batch_loss, elapsed))
        loss.backward()
        optimizer.step()
        scheduler.step()
    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))
    t0 = time.time()
    total_eval_loss = 0
    nb_eval_steps = 0
print("Training complete!")
print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

======== Epoch 1 / 1 ========
Training...
  Batch   500  of  10,189. Loss: 0.4329083561897278.   Elapsed: 0:01:02.
  Batch 1,000  of  10,189. Loss: 0.25056442618370056.   Elapsed: 0:02:03.
  Batch 1,500  of  10,189. Loss: 0.4402444660663605.   Elapsed: 0:03:05.
  Batch 2,000  of  10,189. Loss: 0.1298457384109497.   Elapsed: 0:04:08.
  Batch 2,500  of  10,189. Loss: 0.26451703906059265.   Elapsed: 0:05:10.
  Batch 3,000  of  10,189. Loss: 0.46704205870628357.   Elapsed: 0:06:12.
  Batch 3,500  of  10,189. Loss: 0.17926152050495148.   Elapsed: 0:07:15.
  Batch 4,000  of  10,189. Loss: 0.19153498113155365.   Elapsed: 0:08:17.
  Batch 4,500  of  10,189. Loss: 0.279425710439682.   Elapsed: 0:09:19.
  Batch 5,000  of  10,189. Loss: 0.3570004105567932.   Elapsed: 0:10:21.
  Batch 5,500  of  10,189. Loss: 0.14839531481266022.   Elapsed: 0:11:23.
  Batch 6,000  of  10,189. Loss: 0.15792593359947205.   Elapsed: 0:12:26.
  Batch 6,500  of  10,189. Loss: 0.11067229509353638.   Elapsed: 0:13:28.
  

# Guardar el modelo Entrenado

In [8]:
# Guardando buenas prácticas: si usas los nombres predeterminados para el modelo, puedes recargarlo usando from_pretrained()
output_dir = './model_gpt_ibai/'

# Crear el directorio de salida si es necesario
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Guardando modelo en %s" % output_dir)

# Guarda un modelo entrenado, su configuración y el tokenizador usando `save_pretrained()`.
# Luego pueden recargarse usando `from_pretrained()`
model_to_save = model.module if hasattr(model, 'module') else model  # Manejar entrenamiento distribuido/paralelo
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

Guardando modelo en ./model_gpt_ibai/


('./model_gpt_ibai/tokenizer_config.json',
 './model_gpt_ibai/special_tokens_map.json',
 './model_gpt_ibai/vocab.json',
 './model_gpt_ibai/merges.txt',
 './model_gpt_ibai/added_tokens.json',
 './model_gpt_ibai/tokenizer.json')

# Generación de Texto con el nuevo Modelo

In [9]:
model.eval()

prompt = "<|startoftext|>" + "<|"+control_code+"|>" + "para mi la vida es"

generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
generated = generated.to(device)
#print(generated)

sample_outputs = model.generate(
                                generated, 
                                do_sample=True,   
                                top_k=50, 
                                max_length = 150,
                                top_p=0.95, 
                                num_return_sequences=8
                                )

for i, sample_output in enumerate(sample_outputs):
  print("{}: {}\n\n".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


0: <|ibai|>para mi la vida es una película espectacular.  que no me lo esperaba.  que no me lo esperaba.  si fuera un clásico, ¿ dónde me gustaría ir a ir ? creo que iría de la manera más cómoda. 


1: <|ibai|>para mi la vida es lo que más me molesta.  tengo una vida en la que tengo un montón de cosas que no me pueden valer.  tengo unas ideas, un estilo de vida que estoy haciendo totalmente diferente a lo que estoy haciendo ahora mismo.  llevo ya dos años en la empresa y estoy totalmente de acuerdo con mi jefa, mi mejor amigo y mi mejor aliado. 


2: <|ibai|>para mi la vida es una puta mierda.  y los perros también.  los perros son asesinos, la verdad.  ¿ qué pasa ? de hecho, yo tengo una perra que se llama giant. 


3: <|ibai|>para mi la vida es lo que he hecho.  y la vida también es como mi hija, que es lo que intento con la vida.  mi hija a mí me gusta mucho que tenga su espacio, me guste su espacio y, a veces, no le doy ni tiempo ni espacio a mi familia y no me entero.  y cuando vo

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Ruta del modelo entrenado
model_path = "./model_gpt_ibai"

# Cargar el modelo y el tokenizador
print("Cargando el modelo...")
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)
model.eval()  # Establecer en modo evaluación
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Parámetros de generación
def generate_response(prompt, max_length=150, top_k=30, top_p=0.85):
    """
    Genera una respuesta a partir del prompt dado.
    """
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    output = model.generate(
        inputs['input_ids'],
        max_length=max_length,
        top_k=top_k,
        top_p=top_p,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    # Retornar solo la parte generada (sin el prompt inicial)
    return response[len(prompt):].strip()

# Validar respuestas generadas
def is_valid_response(response):
    """
    Filtra respuestas incoherentes o irrelevantes.
    """
    invalid_keywords = ["puta madre", "cago", "irrelevante", "sin sentido"]
    for word in invalid_keywords:
        if word in response.lower():
            return False
    return True

# Chatbox interactivo
print("¡Bienvenido al BootcampGPT del Bootcamp de Ciencia de Datos & IA.!")
print("Escribe 'salir' para terminar la conversación.\n")

# Contexto inicial
context = "<|startoftext|><|ibai|> Esto es una conversación con Ibai. Él responde preguntas y ayuda en lo que puede.\n"

while True:
    user_input = input("Tú: ")
    if user_input.lower() in ["salir", "exit", "quit"]:
        print("ChatGPT: ¡Adiós!")
        break
    
    # Crear el prompt con contexto
    prompt = context + f"Tú: {user_input}\n<|ibai|> "
    
    # Generar respuesta
    response = generate_response(prompt)
    
    # Validar respuesta; regenerar si es inválida
    while not is_valid_response(response):
        response = generate_response(prompt)
    
    # Mostrar respuesta
    print(f"ChatGPT: {response}")


Cargando el modelo...
¡Bienvenido al BootcampGPT del Bootcamp de Ciencia de Datos & IA.!
Escribe 'salir' para terminar la conversación.

Tú: conoces al kun aguero?
ChatGPT: e es muy fan de kun, me dicen que me ha hecho mucho reír con él.  me ha hecho reír de todo, me dice que me está escuchando.  sí, pero es que me hace reír de todo.  ¿ qué es lo que me hace reír ? no sé si le da risa o qué es lo que me hace reír.
Tú: hola conoces al kun aguero?
ChatGPT: a que hace cosas de las que se puede sentir orgulloso.  no solo a nivel de periodismo, sino en otros muchos ámbitos.  ha sido un profesional que ha pasado por muchas personas, que han cambiado su vida, su forma de ser, de ver el fútbol.  es un periodista que ha cambiado la vida de muchos, de muchos.
